In [12]:
import glob
import argparse
import numpy as np
import os, sys
from os import listdir, makedirs
import shutil
import cv2

import h5py
import io
from PIL import Image

POSDIM = 21
FACEDIM = 70
HANDDIM = 21

def reset(reset_path):
    path = reset_path
    if os.path.isdir(path):
        shutil.rmtree(path)
        print("remove existing "+path)
        makedirs(path)
        print("create folder: "+path)
    else:
        makedirs(path)
        print("create foder: "+path)

# data path
img_h5 = "/Users/shihyaolin/Documents/data/scoliosis/jpgs/20190730_0_5.walk_jpg.h5"
keypoint_h5 = "/Users/shihyaolin/Documents/data/scoliosis/reid_kjsons/20190730_0_5.walk_reid_json.h5"

output = "/Users/shihyaolin/Documents/data/scoliosis/cropped/20190730_0_5.walk/"
reset(output)


remove existing /Users/shihyaolin/Documents/data/scoliosis/cropped/20190730_0_5.walk/
create folder: /Users/shihyaolin/Documents/data/scoliosis/cropped/20190730_0_5.walk/


In [2]:
# show image
f = h5py.File(img_h5, 'r')
print(f.keys())
imgs = f['binary_jpg']
pil_img_obj = Image.open(io.BytesIO(imgs[0]))
pil_img_obj.show()

# show keypoint
f = h5py.File(keypoint_h5, 'r')
print(f.keys())
keypoint = f['keypoint']
print(keypoint[100][0:POSDIM])

<KeysViewHDF5 ['binary_jpg']>
<KeysViewHDF5 ['keypoint']>
[[1.46323e+02 6.55830e+01 8.56680e-01]
 [1.69885e+02 8.28319e+01 7.37097e-01]
 [1.73040e+02 7.89309e+01 6.91392e-01]
 [1.77709e+02 1.11863e+02 1.71986e-01]
 [1.58122e+02 1.29873e+02 4.92142e-01]
 [1.65986e+02 8.51881e+01 8.65961e-01]
 [1.65932e+02 1.14201e+02 8.69350e-01]
 [1.39283e+02 1.22042e+02 8.56417e-01]
 [1.72243e+02 1.41661e+02 6.30139e-01]
 [1.73015e+02 1.41645e+02 5.11463e-01]
 [1.60482e+02 1.90239e+02 7.34559e-01]
 [1.68327e+02 2.35745e+02 7.87886e-01]
 [1.72213e+02 1.43213e+02 6.49746e-01]
 [1.64377e+02 1.91040e+02 7.66872e-01]
 [2.02022e+02 2.28682e+02 8.04118e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00]
 [1.46354e+02 5.93478e+01 9.08115e-01]
 [0.00000e+00 0.00000e+00 0.00000e+00]
 [1.58903e+02 5.61888e+01 8.40618e-01]
 [1.93399e+02 2.47469e+02 7.94282e-01]
 [1.98899e+02 2.45167e+02 6.69482e-01]]


In [86]:
# load data
f = h5py.File(img_h5, 'r')
imgs = f['binary_jpg']

img_list = []
for i, img in enumerate(imgs):
    pil_img = Image.open(io.BytesIO(img)).convert('RGB') 
    open_cv_image = np.array(pil_img) 
    # Convert RGB to BGR 
    open_cv_image = open_cv_image[:, :, ::-1].copy() 
    img_list.append(open_cv_image)


# load keypoint
f = h5py.File(keypoint_h5, 'r')
keypoints = f['keypoint']

keypoint_list = []

for i, keypoint in enumerate(keypoints):
    keypoint_list.append(keypoint)
    


In [87]:
# plot center

width = int(abs(keypoint_list[100][8][1]-keypoint_list[100][16][1]))+40

print(width)

cropped_img_list = []
print(len(img_list[0]))
print(len(img_list[0][1]))

for i, img in enumerate(img_list):
    center = (keypoint_list[i][8][0], keypoint_list[i][8][1])
#     cv2.circle(img, center, 10, (0,255,0), -1)

    x_0 = int(center[0]-width)
    y_0 = int(center[1]-width)
    x_1 = int(center[0]+width)
    y_1 = int(center[1]+width)
    
    if x_0 >0 and x_1 < len(img[1]):
        img2 = img[y_0:y_1, x_0:x_1]
#         cv2.imshow("img", img2)
#         cv2.waitKey(1)
        cv2.imwrite(output+"/%04d.jpg"%i,img2)
        

122
288
352


In [82]:
cv2.destroyAllWindows()